In [138]:
import scipy
import pandas as pd
from scipy.stats import chi2_contingency



def rv_from_uni(p): return int(uni() < p)

# ЗАДАНИЕ

* Написать программы, реализующие рассмотренные методы имитации
случайных событий.
* Провести статистические исследования полученных результатов..
* Проверить гипотезы о соответствии полученных результатов требуемым.

In [139]:
from numpy import random

seed = 1
random.seed(1)
uni = random.uniform
n = 10000

# 1) Имитация сложного события(события независимы)

Даны две независимые дискретные случайные величины(события) $A, B. A \sim Bern(p_{A}), B \sim Bern(p_{B})$


In [140]:
P_A = .1
P_B = .9

### Функция генерации события

In [141]:
def complex_event(p): return rv_from_uni(p)

### Генерация данных

In [142]:
A = [complex_event(P_A) for _ in range(n)]
B = [complex_event(P_B) for _ in range(n)]
df = pd.DataFrame({'A': A, 'B': B})
crosstab = pd.crosstab(index=df['A'], columns=df['B'])
crosstab

B,0,1
A,,
0,888,8109
1,106,897


In [4]:
import seaborn as sns

sns.histplot(data=df, x="sepal_length")

AttributeError: module 'seaborn' has no attribute 'histplot'

### Проверка гипотезы

Для проверки гипотезы воспользуемся $Chi2$ тестом на независимость для категориальных данных. В качестве реализации изпользуем chi2_contingency из пакета scipy. В качетсве $H_{0}$ гипотезы будем считать зависимость случайны величин

In [143]:
chi2, p, dof, ex = chi2_contingency(crosstab)
print(f'p-value={p}')

p-value=0.5185954989787251


Т.к. p-value достаточно велико, мы можем отвергунть гипотезу о зависимости СВ.

# 2) Имитация сложного события, состоящего из зависимых событий.

Даны зависимые дискретные случайные величины(события) $A, B. A \sim Bern(p_{A}), B \sim Bern(p_{B}) , P(A|B) = p_{A|B}, P(B|A)=p_{B|A}$

In [189]:
P_A = .5
P_B_A = 0.3

In [190]:
def complex_dependent_event():
    A = rv_from_uni(P_A)
    B = rv_from_uni(P_B_A) if A else rv_from_uni(1 - P_B_A)
    return [A, B]

In [191]:
data = np.array([complex_dependent_event() for _ in range(n)])
df = pd.DataFrame({'A': data[:,0], 'B': data[:,1]})
crosstab = pd.crosstab(index=df['A'], columns=df['B'])
crosstab

B,0,1
A,,
0,1478,3486
1,3512,1524


### Проверка гипотезы

Для проверки гипотезы воспользуемся $Chi2$ тестом на независимость для категориальных данных. В качестве реализации изпользуем chi2_contingency из пакета scipy. В качетсве $H_{0}$ гипотезы будем считать зависимость случайны величин

In [192]:
chi2, p, dof, ex = chi2_contingency(crosstab)
print(f'p-value={p}')

p-value=0.0


$p-value$ стремится к нулю, что позволяет нам отвергнуть нулевую гипотезу. Зависимость СВ доказана

# 3) Имитация событий, составляющих полную группу

Даны три независимые дискретные случайные величины(события) $A, B, С. A \sim Bern(p_{A}), B \sim Bern(p_{B}), C \sim Bern(p_{C})$

In [193]:
P_A = .3
P_B = .3
P_C = .4

In [194]:
#A, B, C = 0, 1, 2
def triple_complex_event():
    t = uni()
    return 0 if t < P_A else 1 if P_A < t < P_A + P_B else 2

In [196]:

data = [triple_complex_event() for _ in range(10)]
df = pd.DataFrame({'state': data})
crosstab = df.groupby('state').size()
crosstab

state
0    3
1    1
2    6
dtype: int64

In [197]:
chi2, p, dof, ex = chi2_contingency(crosstab)
print(f'p-value={p}')

p-value=1.0


$p-value$ стремится к 1, что говорит в пользу нулевой гипотезы о независимости СВ